In [1]:
%reset 

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
## note this forcing data is for the entire CONUS so i need to mask out what isn't in the CRB 
## note: for masking, ufunc functions can be used directly on a DataArray
%reset_selective
%matplotlib inline
import numpy as np
import xray
import os
import math 
basin = "cascades"
model = "CCSM4"
scenario = "historical"

direc = '/raid/gergel/%s' % "tmin"
tmin_file = "%s_%s_%s.nc" % (model,scenario,"tasmin")
tmin = xray.open_dataset(os.path.join(direc,tmin_file),chunks={'time': 10})

direc = '/raid/gergel/%s' % "tmax"
tmax_file = "%s_%s_%s.nc" % (model,scenario,"tasmax")
tmax = xray.open_dataset(os.path.join(direc,tmax_file),chunks={'time': 10})

f_masks = '/raid9/gergel/agg_snowpack/sm_summer/masks.npz'
masks = np.load(f_masks)
gb = masks['gb'].reshape(1,382,346)
lc = masks['lc'].reshape(1,382,346)
mo = masks['mo'].reshape(1,382,346)
ds_lc = xray.DataArray(lc).to_dataset()
ds_lc.rename({"dim_0": "lat", "dim_1": "lon"}, inplace=True)
ds_gb = xray.DataArray(gb).to_dataset()
ds_gb.rename({"dim_0": "lat", "dim_1": "lon"}, inplace=True)
ds_mo = xray.DataArray(mo).to_dataset()
ds_mo.rename({"dim_0": "lat", "dim_1": "lon"}, inplace=True)

############################ masking with swe/no swe ############################## 
swe_mask_file = '/raid9/gergel/agg_snowpack/goodleap/SWE/histmeanmask.nc' ## 1s are swe, 0s are no swe 
swe_mask = xray.open_dataset(swe_mask_file)
###########################################################################

## rename dimensions
swe_mask.rename({"Latitude": "lat", "Longitude": "lon", "Time": "time"}, inplace=True)
# swe_mask[dict("lat" < 180)] = lat + 180
# tmin['lon'][tmin['lon'] > 180] - 360
## adjust lat/lon dimensions since the index names are different
tmin_lons_new = tmin['lon'].values[tmin['lon'].values > 180] - 360 
tmin['lon'] = tmin_lons_new
tmax['lon'] = tmin_lons_new

## deal with time dimension length issue with swe_mask
swe_mask = swe_mask.squeeze()

## Dataset join 
swe_mask_align,tmin_align = xray.align(swe_mask,tmin,join='inner',copy=False)
swe_mask_align,tmax_align = xray.align(swe_mask,tmax,join='inner',copy=False)

## now do masking
if (basin == "cascades"):
    arr_tmin = tmin_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "california"):
    arr_tmin = tmin_align.where(swe_mask.time == 1 and tmin.lat < 34.5 and tmin.lat > 41.5 and
                          tmin.lon < -122.5 and tmin.lon > -117)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "northernrockies"):
    arr_tmin = tmin_align.where(swe_mask.time == 1 and tmin.lat < 42.37 and tmin.lat > 49.0 and
                          tmin.lon < -119.5 and tmin.lon > -107.5)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "southernrockies"): 
    arr_tmin = tmin_align.where(swe_mask.time == 1 and tmin.lat < 35 and tmin.lat > 42.37 and
                          tmin.lon < -113 and tmin.lon > -104)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "whites"):
    arr_tmin = tmin_align.where(swe_mask.time == 1 and tmin.lat < 32 and tmin.lat > 36 and
                          tmin.lon < -111 and tmin.lon > -108)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "nwinterior"):
    arr_tmin = tmin_align.where(swe_mask.time == 0 and tmin.lat < 42 and tmin.lat > 49 and
                          tmin.lon < -121.5 and tmin.lon > -111)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "coastalsouth"):
    arr_tmin = tmin_align.where(swe_mask.time == 0 and tmin.lat < 33 and tmin.lat > 41.5 and
                          tmin.lon < -124 and tmin.lon > -118.5)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "coastalnorth"):
    arr_tmin = tmin_align.where(swe_mask.time == 0 and tmin.lat < 41.5 and tmin.lat > 48 and
                          tmin.lon < -124.5 and tmin.lon > -122.0)
    arr_tmax = tmax_align.where(swe_mask.time == 1 and tmin.lat < 41.5 and tmin.lat > 49.0 and
                          tmin.lon < -123.25 and tmin.lon > -119.5)
elif (basin == "lower_colorado"):
    arr_tmin = tmin_align.where(swe_mask.time == 0 and ds_lc == 1)
    arr_tmax = tmax_align.where(swe_mask.time == 0 and ds_lc == 1)
elif (basin == "missouri"):
    arr_tmin = tmin_align.where(swe_mask.time == 0 and ds_mo == 1)
    arr_tmax = tmax_align.where(swe_mask.time == 0 and ds_lc == 1)
else: 
    arr_tmin = tmin_align.where(swe_mask.time == 0 and ds_gb == 1)
    arr_tmax = tmax_align.where(swe_mask.time == 0 and ds_lc == 1)
    
del swe_mask_align 
del tmin 
del tmax 
del tmax_align 
del tmin_align

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
No regex pattern specified. Nothing done.


ImportError: No module named dask

In [ ]:
## estimate relative humidity using hypsometric equation for pressure, virtual temperature and t_avg
Rd = 287.04 ## J kg^-1 K ^-1
p1 = 100000 ## Pa
T0 = 273.15 ## reference temperature
g = 9.81 ## m / s^2
## need to bring in arr_w
arr_w = arr_tmin
## need to get elevations for grid cells
h = np.ones(shape=(len(arr_tmin.lat),len(arr_tmin.lon)),dtype='float')
t_avg = (arr_tmin.air_temp_min + arr_tmax.air_temp_max) * 0.5
##  Tv = T(1 + 0.61w)
Tv = t_avg * (1 + 0.61*arr_w)
## p = e ^ (-(((z_2 - z_1) * g)/ (Rd*Tv) - ln(p1)))
hg = h * g
RdTv = Rd * Tv

p = np.exp(-((np.divide(hg,RdTv.air_temp_min)) - math.log(p1)))
## RH = 0.263*p*q[exp(17.67*T - T0)/(T - 29.65)]^-1
RH = 0.263*p*arr_w * ((np.exp((17.67 * (t_avg - T0))/ (t_avg - 29.65)))**-1)

del p,RdTv,Tv,hg,arr_w

## estimate vapor pressures
satvpd = np.exp(1.81 + (((t_avg * 17.27) - 4717.31)/(t_avg - 35.86)))
satvpx = np.exp(1.81 + (((arr_tmin.air_temp_min * 17.27) - 
                         4717.31)/(arr_tmin.air_temp_min - 35.86)))
satvpn = np.exp(1.81 + (((arr_tmax.air_temp_max * 17.27) - 
                         4717.31)/(arr_tmax.air_temp_max - 35.86)))
ambvp = (RH * satvpd) * (1.0/100.0)
RHmax = 100.0 * (ambvp/satvpn)
RHmin = 100.0 * (ambvp/satvpx)

## remove extra data objects
del ambvp 
del satvpn 
del satvpx 
del satvpd 

## estimate EMCs
if RH < 10: 
    EMC = 0.03229 + 0.281073*RH - 0.000578*t_avg*RH
elif RH >= 10 and RH <= 50:
    EMC = 2.22749 - 0.160107*RH - 0.014784*t_avg
else: 
    EMC = 21.0606 + (0.005565*(RH**2)) - (0.00035*RH*t_avg) - (0.483199*RH)

if RHmax < 10:
    EMCmax = 0.03229 + 0.281073*RHmax - 0.000578*arr_tmax*RHmax
elif RHmax >= 10 and RHmax <= 50:
    EMCmax = 2.22749 - 0.160107*RHmax - 0.014784*arr_tmax
else: 
    EMCmax = 21.0606 + (0.005565*(RHmax**2)
                       ) - (0.00035*RHmax*arr_tmax) - (0.483199*RHmax)

if RHmin < 10:
    EMCmin = 0.03229 + 0.281073*RHmin - 0.000578*arr_tmin*RHmin
elif RHmin >= 10 and RHmin <= 50:
    EMCmin = 2.22749 - 0.160107*RHmin - 0.014784*arr_tmin
else: 
    EMCmin = 21.0606 + (0.005565*(RHmin**2)
                       ) - (0.00035*RHmin*arr_tmin) - (0.483199*RHmin)


